In [1]:
#####  LOADING AND EXTRACTING DATA   #####


import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
%matplotlib inline
def data_loader2():
    
    # Load dataset file
    data_frame = pd.read_csv('training.csv')
    
    data_frame['Image'] = data_frame['Image'].apply(lambda i: np.fromstring(i, sep=' '))
    data_frame = data_frame.dropna()  # Get only the data with 15 keypoints
   
    # Extract Images pixel values
    imgs_array = np.vstack(data_frame['Image'].values)/ 255.0
    imgs_array = imgs_array.astype(np.float32)    # Normalize, target values to (0, 1)
    imgs_array = imgs_array.reshape(-1, 96, 96, 1)
        
    # Extract labels (key point cords)
    labels_array = data_frame[data_frame.columns[:-1]].values
    labels_array = (labels_array - 48) / 48    # Normalize, traget cordinates to (-1, 1)
    labels_array = labels_array.astype(np.float32) 
    
    # shuffle the train data
#     imgs_array, labels_array = shuffle(imgs_array, labels_array, random_state=9)  
    
    return imgs_array, labels_array

def data_loader(path):
    data_frame = pd.read_csv(path)
    # Extract images
    data_frame['image'] = data_frame['image'].apply(lambda i: np.fromstring(i, sep=' '))
    data_frame = data_frame.dropna()  # Get only the data with 15 keypoints
    imgs_array = np.vstack(data_frame['image'].values)/255.0
    imgs_array = imgs_array.astype(np.float32)    # Normalize, target values to (0, 1)
    imgs_array = imgs_array.reshape(-1, 128, 128, 1)
    # Extract labels (key point cords)
    labels_array = data_frame[data_frame.columns[:-1]].values
#     labels_array = (labels_array - 64) / 64    # Normalize, traget cordinates to (-1, 1)
    labels_array = (labels_array)     # Normalize, traget cordinates to (-1, 1)
    labels_array = labels_array.astype(np.float32)
#     shuffle the train data
    imgs_array, labels_array = shuffle(imgs_array, labels_array, random_state=9)  
    return imgs_array,labels_array





In [2]:
# n=7
# labels[n] = (labels[n]*48)+48
# print(labels[n])
# image = np.squeeze(imgs[n])
# plt.imshow(image, cmap='gray')
# plt.plot(labels[n][::2], labels[n][1::2], 'ro')
# plt.show()



In [3]:
######   BUILD, TRAIN AND SAVE THE CONVOLUTIONAL MODEL    ########


from tensorflow.keras.models import Sequential
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.layers import Flatten, Dense
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.callbacks import ModelCheckpoint, History
from tensorflow.keras.regularizers import l2
# from keras.optimizers import Adam
# X_train.shape

# Main model
def the_model():
    model = Sequential()
    
    # Layer 1 
    model.add(Conv2D(32, (3,3),padding='same',activation='relu',input_shape=X_train.shape[1:])) 
    model.add(MaxPooling2D(pool_size=2)) ## 64*64
    
    # Layer 2
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2)) ## 32*32
    
    # Layer 3
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2)) ## 16*16
    
    # Layer 4
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2)) ## 8*8
    
    # Layer 5
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2)) ## 4*4
    
    # Layer 6
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2)) ## 2*2
    
    # Layer 7
    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(2048, activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(196)) ## 68 facial landmarks points
    
    return model









In [4]:
X_train, y_train = data_loader("training_sets/train_98_2.csv")
print("Training datapoint shape: X_train.shape:{}".format(X_train.shape))
print("Training labels shape: y_train.shape:{}".format(y_train.shape))

Training datapoint shape: X_train.shape:(10000, 128, 128, 1)
Training labels shape: y_train.shape:(10000, 196)


In [5]:
batch_size = 64

model = the_model()
hist = History()

checkpointer = ModelCheckpoint(filepath='checkpoints/checkpoint_128_9.hdf5', 
                               verbose=1, save_best_only=True)

In [6]:
# from datetime import datetime
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# file_writer = tf.summary.create_file_writer(logdir + "/metrics")
# file_writer.set_as_default()
import tensorflow.keras
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Complie Model
epochs = 500
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model_fit = model.fit(X_train, y_train, validation_split=0.30, epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist], verbose=1)

model.save('models/model_128_9.h5')
print("Save model successfully")

Train on 7000 samples, validate on 3000 samples
Epoch 1/500
 640/7000 [=>............................] - ETA: 2:37 - loss: 27926.1087 - accuracy: 0.0234

In [ ]:
#####  LOADING AND EXTRACTING DATA   #####


import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
%matplotlib inline
def data_loader():
    
    # Load dataset file
    data_frame = pd.read_csv('training.csv')
    
    data_frame['Image'] = data_frame['Image'].apply(lambda i: np.fromstring(i, sep=' '))
    data_frame = data_frame.dropna()  # Get only the data with 15 keypoints
   
    # Extract Images pixel values
    imgs_array = np.vstack(data_frame['Image'].values)/ 255.0
    imgs_array = imgs_array.astype(np.float32)    # Normalize, target values to (0, 1)
    imgs_array = imgs_array.reshape(-1, 96, 96, 1)
        
    # Extract labels (key point cords)
    labels_array = data_frame[data_frame.columns[:-1]].values
    labels_array = (labels_array - 48) / 48    # Normalize, traget cordinates to (-1, 1)
    labels_array = labels_array.astype(np.float32) 
    
    # shuffle the train data
#     imgs_array, labels_array = shuffle(imgs_array, labels_array, random_state=9)  
    
    return imgs_array, labels_array

  


# This snippet is just to check/verify data
imgs, labels = data_loader()
print(imgs.shape)
print(labels.shape)

n=0
labels[n] = (labels[n]*48)+48
image = np.squeeze(imgs[n])
plt.imshow(image, cmap='gray')
plt.plot(labels[n][::2], labels[n][1::2], 'ro')
plt.show()

In [ ]:
X_train, y_train = data_loader()
print("Training datapoint shape: X_train.shape:{}".format(X_train.shape))
print("Training labels shape: y_train.shape:{}".format(y_train.shape))

In [ ]:
def the_model():
    model = Sequential()
    
    model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=X_train.shape[1:])) # Input shape: (96, 96, 1)
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    # Convert all values to 1D array
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(30))
    
    return model
batch_size = 64

model = the_model()
hist = History()

checkpointer = ModelCheckpoint(filepath='checkpoint1.hdf5', 
                               verbose=1, save_best_only=True)

In [ ]:
# Complie Model
epochs = 40
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model_fit = model.fit(imgs, labels, validation_split=0.2, epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist], verbose=1)

model.save('model.h5')
print("Save model successfully")